## Data input for BIDS datasets
`DataGrabber` and `SelectFiles` are great if you are dealing with generic datasets with arbitrary organization. However if you have decided to use Brain Imaging Data Structure (BIDS) to organized your data (or got your hands on a BIDS dataset) you can take advanted of a formal structure BIDS imposes. In this short tutorial you will learn how to do this.

## `pybids` - a Python API for working with BIDS datasets
`pybids` is a lightweight python API for querying BIDS folder structure for specific files and metadata. You can install it from PyPi:
```
pip install bids
```
Please note it should be already installed in the tutorial Docker image.

## The `layout` object and simple queries
To begin working with pubids we need to initalize a layout object. We will need it to do all of our queries

In [1]:
from bids.grabbids import BIDSLayout
layout = BIDSLayout("/data/ds102/")

Let's figure out what are the subject labels in this dataset

In [2]:
layout.get_subjects()

['01', '02', '03', '04', '05']

What modalities are included in this dataset?

In [3]:
layout.get_modalities()

['anat', 'func']

What different data types are included in this dataset?

In [4]:
layout.get_types()

['bold', 'events', 'T1w']

In [5]:
layout.get_types(modality='func')

['bold', 'events']

What are the different tasks included in this dataset?

In [6]:
layout.get_tasks()

['flanker']

We can also ask for all of the data for a particular subject.

In [7]:
layout.get(subject='01')

[File(filename='/data/ds102/sub-01/anat/sub-01_T1w.nii.gz', type='T1w', modality='anat', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', type='bold', task='flanker', run='1', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-1_events.tsv', type='events', task='flanker', run='1', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz', type='bold', task='flanker', run='2', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-2_events.tsv', type='events', task='flanker', run='2', modality='func', subject='01')]

We can also ask for a specific subset of data. Note that we are using extension filter to get just the imaging data (BIDS allows both .nii and .nii.gz so we need to include both).

In [8]:
layout.get(subject='01', type='bold', extensions=['nii', 'nii.gz'])

[File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', type='bold', task='flanker', run='1', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz', type='bold', task='flanker', run='2', modality='func', subject='01')]

You probably noticed that this method does not only return the file paths, but objects with relevant query fields. We can easily extract just the file paths.

In [9]:
[f.filename for f in layout.get(subject='01', type='T1w', extensions=['nii', 'nii.gz'])]

['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz']

## Including `pybids` in your `nipype` workflow
This is great, but what we really want is to include this into our `nipype` workflows. How to do this? We can create our own custom `BIDSDataGrabber` using a `Function` Interface. First we need a plain Python function that for a given subject label and dataset location will return list of BOLD and T1w files.

In [10]:
def get_niftis(subject_id, data_dir):
    # Remember that all the necesary imports need to be INSIDE the function for the Function Interface to work!
    from bids.grabbids import BIDSLayout
    
    layout = BIDSLayout(data_dir)
    
    bolds = [f.filename for f in layout.get(subject=subject_id, type='bold', extensions=['nii', 'nii.gz'])]
    T1ws = [f.filename for f in layout.get(subject=subject_id, type='T1w', extensions=['nii', 'nii.gz'])]
    
    return bolds, T1ws

In [11]:
get_niftis('01', '/data/ds102')

(['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz',
  '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz'],
 ['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz'])

Ok we got our function. Now we need to wrap it inside a Node object.

In [12]:
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import IdentityInterface, Function

In [13]:
BIDSDataGrabber = Node(Function(function=get_niftis, input_names=["subject_id",
                                       "data_dir"],
                                   output_names=["bolds", 
                                        "T1ws"]), name="BIDSDataGrabber")
BIDSDataGrabber.inputs.data_dir = "/data/ds102"

In [14]:
BIDSDataGrabber.inputs.subject_id='01'
res = BIDSDataGrabber.run()
res.outputs

170324-16:47:04,997 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmpsAYO9O/BIDSDataGrabber



T1ws = ['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz']
bolds = ['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz']

Works like a charm! (hopefully :) Lets put it in a workflow. We are not going to analyze any data, but for demostrantion purposes we will add a couple of nodes that pretend to analyze their inputs

In [15]:
def printMe(paths):
    print("\n\nanalyzing " + str(paths) + "\n\n")
    
analyzeBOLD = Node(Function(function=printMe, input_names=["paths"],
                                   output_names=[]), name="analyzeBOLD")

analyzeT1w = Node(Function(function=printMe, input_names=["paths"],
                                   output_names=[]), name="analyzeT1w")


In [16]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD, "paths")
wf.connect(BIDSDataGrabber, "T1ws", analyzeT1w, "paths")
wf.run()

170324-16:47:05,273 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170324-16:47:05,437 workflow INFO:
	 Running serially.
170324-16:47:05,438 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmpE9XT25/bids_demo/BIDSDataGrabber
170324-16:47:05,483 workflow INFO:
	 Executing node analyzeBOLD in dir: /tmp/tmpfQYORd/bids_demo/analyzeBOLD


analyzing ['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz']


170324-16:47:05,513 workflow INFO:
	 Executing node analyzeT1w in dir: /tmp/tmp8YkknY/bids_demo/analyzeT1w


analyzing ['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz']




## Iterating over subject labels
In the previous example we demostrated how to use `pybids` to "analyze" one subject. How can we scale it for all subjects? Easy - using `iterables`.

In [17]:
BIDSDataGrabber.iterables = ('subject_id', layout.get_subjects())
wf.run()

170324-16:47:05,572 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170324-16:47:05,753 workflow INFO:
	 Running serially.
170324-16:47:05,757 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a3 in dir: /tmp/tmp8BCLDf/bids_demo/_subject_id_04/BIDSDataGrabber
170324-16:47:05,826 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a0 in dir: /tmp/tmpUxqmKs/bids_demo/_subject_id_01/BIDSDataGrabber
170324-16:47:05,896 workflow INFO:
	 Executing node analyzeBOLD.a0 in dir: /tmp/tmp8SXshs/bids_demo/_subject_id_01/analyzeBOLD


analyzing ['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz']


170324-16:47:05,920 workflow INFO:
	 Executing node analyzeBOLD.a3 in dir: /tmp/tmpQk6EGe/bids_demo/_subject_id_04/analyzeBOLD


analyzing ['/data/ds102/sub-04/func/sub-04_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-04/func/sub-04_task-flanker_run-2_bold.nii.gz']


170324-16:47:05,975 workf

## Accessing additional metadata
Querying different files is nice, but sometimes you want to access more metadata. For example `RepetitionTime`. `pybids` can help with that as well

In [18]:
layout.get_metadata('/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz')

{u'CogAtlasID': u'tsk_4a57abb949a4f',
 u'EchoTime': 0.03,
 u'FlipAngle': 80.0,
 u'MRAcquisitionType': u'2D',
 u'MagneticFieldStrength': 3.0,
 u'Manufacturer': u'Siemens',
 u'ManufacturerModelName': u'Allegra',
 u'RepetitionTime': 2.0,
 u'ScanningSequence': u'Echo Planar',
 u'TaskDescription': u'On each trial (inter-trial interval (ITI) varied between 8 s and 14 s; mean ITI=12 s),participants used one of two buttons on a response pad to indicate the direction of a central arrow in an array of 5 arrows. In congruent trials the flanking arrows pointed in the same direction as the central arrow (e.g., < < < < <), while in more demanding incongruent trials the flanking arrows pointed in the opposite direction (e.g., < < > < <). Subjects performed two 5-minute blocks, each containing 12 congruent and 12 incongruent trials, presented in a pseudorandom order.',
 u'TaskName': u'Flanker'}

Can we incorporate this into our pipeline? Yes we can!

In [19]:
def printMetadata(path, data_dir):
    from bids.grabbids import BIDSLayout
    layout = BIDSLayout(data_dir)
    print("\n\nanalyzing " + path + "\nTR: "+ str(layout.get_metadata(path)["RepetitionTime"]) + "\n\n")
    
analyzeBOLD2 = MapNode(Function(function=printMetadata, input_names=["path", "data_dir"],
                             output_names=[]), name="analyzeBOLD2", iterfield="path")
analyzeBOLD2.inputs.data_dir = "/data/ds102/"

In [20]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD2, "path")
wf.connect(BIDSDataGrabber, "T1ws", analyzeT1w, "paths")
wf.run()

170324-16:47:06,580 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170324-16:47:06,772 workflow INFO:
	 Running serially.
170324-16:47:06,776 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a3 in dir: /tmp/tmpcgwNa6/bids_demo/_subject_id_04/BIDSDataGrabber
170324-16:47:06,862 workflow INFO:
	 Executing node analyzeBOLD2.a3 in dir: /tmp/tmpMj_ARH/bids_demo/_subject_id_04/analyzeBOLD2
170324-16:47:06,886 workflow INFO:
	 Executing node _analyzeBOLD20 in dir: /tmp/tmpMj_ARH/bids_demo/_subject_id_04/analyzeBOLD2/mapflow/_analyzeBOLD20


analyzing /data/ds102/sub-04/func/sub-04_task-flanker_run-1_bold.nii.gz
TR: 2.0


170324-16:47:06,930 workflow INFO:
	 Executing node _analyzeBOLD21 in dir: /tmp/tmpMj_ARH/bids_demo/_subject_id_04/analyzeBOLD2/mapflow/_analyzeBOLD21


analyzing /data/ds102/sub-04/func/sub-04_task-flanker_run-2_bold.nii.gz
TR: 2.0


170324-16:47:06,992 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a0 in dir: /tmp/tmpSCLTf6/bids_demo/_